In [8]:
from time import sleep
from datetime import datetime
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import os
import pickle
import ast
import shap
import matplotlib.pyplot as plt
from hypermapper import optimizer
from joblib import dump, load
%matplotlib inline

In [2]:
df = pd.read_csv('../../NetML_nPrints_AMIR.csv')

In [3]:
df['feat_netml'] = df['feat_netml'].apply(lambda x: np.fromstring(x.replace("[", "").replace("]", "").replace("\n", " "), sep=' '))
df['device'] = df['device'].astype('category').cat.codes

In [4]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_X_train = np.vstack(df_train['feat_netml'].values)
df_Y_train = df_train[['device', 'user']]
df_X_test = np.vstack(df_test['feat_netml'].values)
df_Y_test = df_test[['device', 'user']]

In [5]:
selected_columns = [1, 5, 6, 7, 9, 17, 19, 20, 23, 24, 25, 27, 34, 35]
print(len(selected_columns))
df_X_train = df_X_train[:, selected_columns]
df_X_test = df_X_test[:, selected_columns]

14


In [6]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score
from joblib import dump, load
def get_accuracy_for_feat_subset(df_X_train, df_X_test, df_Y_train, df_Y_test, feat_idx, custom_text_ext):
    print(feat_idx)
    if not feat_idx:
        return 1, 0.00001
    new_train = df_X_train[:, feat_idx]
    new_test = df_X_test[:, feat_idx]
    threat_model = RandomForestClassifier(random_state=42)
    threat_model.fit(new_train, df_Y_train.iloc[:, 1].to_numpy())
    source_model = RandomForestClassifier(random_state=42)
    source_model.fit(new_train, df_Y_train.iloc[:, 0].to_numpy())
    threat_pred_new = threat_model.predict(new_test)
    source_pred_new = source_model.predict(new_test)
    threat_score = accuracy_score(df_Y_test.iloc[:, 1].to_numpy(), threat_pred_new)  # Source IP re-id classifier (Threat)
    source_score = accuracy_score(df_Y_test.iloc[:, 0].to_numpy(), source_pred_new) # Network Intrusion classifier
    return threat_score, source_score

In [ ]:
parameters_file = "hypermapper-param-iot_nprint_3pkts.json"
def bayesian_opt(X):
    feat_idx = []
    for i in range(1, 15):
        if X[f'f{i}'] == 'true':
            feat_idx.append(i-1)
    threat_score, source_score = get_accuracy_for_feat_subset(df_X_train, df_X_test, df_Y_train, df_Y_test, feat_idx, 'bayesian')
    print("Source accuracy and threat accuracy:", source_score, threat_score)
    return threat_score/source_score
value_at_optimum = optimizer.optimize(parameters_file, bayesian_opt)

Design of experiment phase, number of new doe samples = 16384 .......
[1, 9, 10]
Source accuracy and threat accuracy: 0.9373695198329853 0.5427974947807933
[]
Source accuracy and threat accuracy: 1e-05 1
[13]
Source accuracy and threat accuracy: 0.7390396659707724 0.34029227557411273
[12]
Source accuracy and threat accuracy: 0.5908141962421712 0.34029227557411273
[12, 13]
Source accuracy and threat accuracy: 0.8183716075156576 0.38204592901878914
[11]
Source accuracy and threat accuracy: 0.9081419624217119 0.48851774530271397
[11, 13]
Source accuracy and threat accuracy: 0.918580375782881 0.5052192066805845
[11, 12]
Source accuracy and threat accuracy: 0.8643006263048016 0.4697286012526096
[11, 12, 13]
Source accuracy and threat accuracy: 0.906054279749478 0.511482254697286
[10]
Source accuracy and threat accuracy: 0.826722338204593 0.4780793319415449
[10, 13]
Source accuracy and threat accuracy: 0.9227557411273486 0.5260960334029228
[10, 12]
Source accuracy and threat accuracy: 0.8371